In [44]:
import gym
import numpy as np
import random
import time

In [3]:
# Initialize Q-value table randomly
q_table = np.zeros((env.observation_space.n, env.action_space.n))
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [5]:
def q_learning(env, num_episodes, num_steps_per_episode, learning_rate, gamma, max_epsilon, min_epsilon, epsilon_decay_rate):
    q_table = np.zeros((env.observation_space.n, env.action_space.n))
    rewards_all = []
    for episode in range(num_episodes):
        state = env.reset()

        reward_episode = 0.0
        done = False
        epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-epsilon_decay_rate*episode)
        for step in range(num_steps_per_episode):
            exploration = random.uniform(0,1)
            if exploration < epsilon:
                action = env.action_space.sample()
            else:
                action = np.argmax(q_table[state, :])

            next_state, reward, done, info = env.step(action)
            q_table[state, action] = q_table[state, action] * (1 - learning_rate) + learning_rate * (reward + gamma * np.max(q_table[next_state,:]))

            reward_episode += reward
            state = next_state

            if done:
                break
        rewards_all.append(reward_episode)
    print(f'Episode {episode} finished')
    return q_table, rewards_all

# Sarsa

In [6]:
def choose_action(env,epsilon,state,q_table,):
  exploration = random.uniform(0,1)
  if exploration < epsilon:
      action = env.action_space.sample()
  else:
      action = np.argmax(q_table[state, :])
  return action

In [7]:
def sarsa(env, num_episodes, num_steps_per_episode, learning_rate, gamma, max_epsilon, min_epsilon, epsilon_decay_rate):
  q_table = np.zeros((env.observation_space.n, env.action_space.n))
  rewards_all = []
  for episode in range(num_episodes):
      state_1 = env.reset()

      reward_episode = 0.0
      done = False
      epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-epsilon_decay_rate*episode)
      action_1 = choose_action(env,epsilon,state_1,q_table)
      for step in range(num_steps_per_episode):
        state_2, reward, done, info = env.step(action_1)
        action_2 = choose_action(env,epsilon,state_2,q_table)

        q_table[state_1, action_1] = q_table[state_1, action_1] + learning_rate * (reward + gamma * q_table[state_2, action_2] - q_table[state_1, action_1])

        state_1 = state_2
        action_1 = action_2

        reward_episode += reward

        if done:
              break
      rewards_all.append(reward_episode)
  print(f'Episode {episode} finished')
  return q_table, rewards_all


In [21]:
def play(env, q_table, render=False):
    state = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action = np.argmax(q_table[state, :])
        next_state, reward, done, info = env.step(action)
        total_reward += reward
        steps += 1
        if render:
            env.render()
            time.sleep(0.2)
            if not done:
                display.clear_output(wait=True)
        state = next_state

    return (total_reward, steps)

In [22]:
def play_multiple_times(env, q_table, max_episodes):
    success = 0
    list_of_steps = []
    for i in range(max_episodes):
        total_reward, steps = play(env, q_table)

        if total_reward > 0:
            success += 1
            list_of_steps.append(steps)

    print(f'Number of successes: {success}/{max_episodes}')
    print(f'Average number of steps: {np.mean(list_of_steps)}')

# Env FrozenLake-v0

In [27]:
env = gym.make('FrozenLake-v0')

In [28]:
# Hyperparameters
gamma = 0.99
learning_rate = 0.1
max_epsilon = 1.0
min_epsilon = 0.01
epsilon_decay_rate = 0.005

num_episodes = 20000
num_steps_per_episode = 100

In [29]:
q_table, rewards_all = q_learning(env, num_episodes, num_steps_per_episode, learning_rate, gamma, max_epsilon, min_epsilon, epsilon_decay_rate)
play_multiple_times(env, q_table, 1000)

Episode 19999 finished
Number of successes: 713/1000
Average number of steps: 37.842917251051894


In [31]:
q_table_sar, rewards_all_sar = sarsa(env, num_episodes, num_steps_per_episode, learning_rate, gamma, max_epsilon, min_epsilon, epsilon_decay_rate)
play_multiple_times(env, q_table_sar, 1000)

Episode 19999 finished
Number of successes: 758/1000
Average number of steps: 38.66094986807388


# Env FrozenLake8x8-v0

In [32]:
env = gym.make('FrozenLake8x8-v0')

In [39]:
# Hyperparameters
gamma = 0.99
learning_rate = 0.1
max_epsilon = 1.0
min_epsilon = 0.01
epsilon_decay_rate = 0.00005

num_episodes = 20000
num_steps_per_episode = 400

In [40]:
#q_learning
q_table, rewards_all = q_learning(env, num_episodes, num_steps_per_episode, learning_rate, gamma, max_epsilon, min_epsilon, epsilon_decay_rate)
play_multiple_times(env, q_table, 1000)

Episode 19999 finished
Number of successes: 857/1000
Average number of steps: 97.34305717619603


In [43]:
#Sarsa
q_table_sar, rewards_all_sar = sarsa(env, num_episodes, num_steps_per_episode, learning_rate, gamma, max_epsilon, min_epsilon, epsilon_decay_rate)
play_multiple_times(env, q_table_sar, 1000)

Episode 19999 finished
Number of successes: 772/1000
Average number of steps: 80.48056994818653


# Env Taxi-v3

In [45]:
env = gym.make('Taxi-v3')

In [46]:
# Hyperparameters
gamma = 0.99
learning_rate = 0.1
max_epsilon = 1.0
min_epsilon = 0.01
epsilon_decay_rate = 0.005

num_episodes = 20000
num_steps_per_episode = 100

In [47]:
#q_learning
q_table, rewards_all = q_learning(env, num_episodes, num_steps_per_episode, learning_rate, gamma, max_epsilon, min_epsilon, epsilon_decay_rate)
play_multiple_times(env, q_table, 1000)

Episode 19999 finished
Number of successes: 1000/1000
Average number of steps: 13.136


In [48]:
#Sarsa
q_table_sar, rewards_all_sar = sarsa(env, num_episodes, num_steps_per_episode, learning_rate, gamma, max_epsilon, min_epsilon, epsilon_decay_rate)
play_multiple_times(env, q_table_sar, 1000)

Episode 19999 finished
Number of successes: 1000/1000
Average number of steps: 13.12


# Nhận xét
Q_learning và SarSa hoạt động ổn trên môi trường FrozenLake-v0 với số lượt thắng khá cao và SarSa có vẻ hoạt động tốt hơn Q_learning trên môi trường này với số lượt thắng lớn hơn với trung bình các bước đi tuy lớn hơn Q_learning nhưng không đáng kể chỉ xấp xỉ 1 bước đi. Nhưng đến với môi trường FrozenLake8x8-v0 có thể thấy môi trường này lớn gấp 4 lần môi trường FrozenLake-v0 và cả 2 thuật toán đều sẽ không hoạt động tốt với không gian trạng thái quá lớn, chính vì thế phải điểu chỉnh hyperparameters phù hợp (num_steps mỗi episode cao hơn, epsilon_decay_rate thấp). Ở môi trường này thì có vẻ Q_learning hoạt động tốt hơn với số lần chiến thắng nhiều hơn SarSa nhưng số bước đi trung bình để đạt chiến thắng khá cao. Đến với môi trường cuối cung Taxi-v3 thì cả 2 thuật toán đều hoạt động rất tốt với số lượt thắng tuyết đối và số bước đi trung bình của cả 2 thuật toán xấp xỉ nhau. 